In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine, inspect
from dotenv import load_dotenv

In [ ]:
load_dotenv()

USER = os.getenv("POSTGRES_USER")
PASSWORD = os.getenv("POSTGRES_PASSWORD")
PORT = os.getenv("POSTGRES_PORT")
HOST = os.getenv("HOST")
DB_NAME = os.getenv("POSTGRES_DB")
PATH_ROOT=os.getenv("PATH_ROOT")
    
engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}")

print(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}")

In [ ]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT table_schema, table_name
        FROM information_schema.tables
        WHERE table_schema = 'silver'
          AND table_name = 'cartola_lh';
    """))
    rows = result.fetchall()

if rows:
    print(" Tabela silver.cartola_lh encontrada:", rows)
else:
    print(" Tabela silver.cartola_lh NÃO encontrada")


In [ ]:
from sqlalchemy import text

with engine.begin() as conn:
    conn.execute(text("CREATE SCHEMA IF NOT EXISTS gold;"))


In [ ]:
from sqlalchemy import text

ddl_dim_atleta = """
DROP TABLE IF EXISTS gold.dim_atleta CASCADE;

CREATE TABLE gold.dim_atleta (
    atleta_sk           BIGSERIAL PRIMARY KEY,
    atleta_id           BIGINT,
    atleta_apelido      TEXT,
    atleta_posicao_id   BIGINT,
    atleta_posicao      TEXT
);

INSERT INTO gold.dim_atleta (atleta_id, atleta_apelido, atleta_posicao_id, atleta_posicao)
SELECT DISTINCT
    atleta_id,
    atleta_apelido,
    atleta_posicao_id,
    atleta_posicao
FROM silver.cartola_lh;

"""

with engine.begin() as conn:
    conn.execute(text(ddl_dim_atleta))

print("gold.dim_atleta criada/populada")


In [ ]:
ddl_dim_clube = """
DROP TABLE IF EXISTS gold.dim_clube CASCADE;

CREATE TABLE gold.dim_clube (
    clube_sk         BIGSERIAL PRIMARY KEY,
    clube_id         BIGINT,
    clube_nome       TEXT,
    clube_abreviacao TEXT,
    clube_slug       TEXT
);

INSERT INTO gold.dim_clube (clube_id, clube_nome, clube_abreviacao, clube_slug)
SELECT DISTINCT
    clube_id,
    clube_nome,
    clube_abreviacao,
    clube_slug
FROM silver.cartola_lh;


"""

with engine.begin() as conn:
    conn.execute(text(ddl_dim_clube))

print("gold.dim_clube criada/populada")


In [ ]:
ddl_dim_partida = """
DROP TABLE IF EXISTS gold.dim_partida CASCADE;

CREATE TABLE gold.dim_partida (
    partida_sk                     BIGSERIAL PRIMARY KEY,
    partida_id                     BIGINT,
    temporada                      INTEGER,
    partida_rodada                 BIGINT,
    partida_clube_casa_id          BIGINT,
    partida_clube_mandante_nome    TEXT,
    partida_clube_visitante_id     BIGINT,
    partida_clube_visitante_nome   TEXT,
    partida_placar_mandante        DOUBLE PRECISION,
    partida_placar_visitante       DOUBLE PRECISION
);

INSERT INTO gold.dim_partida (
    partida_id,
    temporada,
    partida_rodada,
    partida_clube_casa_id,
    partida_clube_mandante_nome,
    partida_clube_visitante_id,
    partida_clube_visitante_nome,
    partida_placar_mandante,
    partida_placar_visitante
)
SELECT DISTINCT
    partida_id,
    temporada,
    partida_rodada,
    partida_clube_casa_id,
    partida_clube_mandante_nome,
    partida_clube_visitante_id,
    partida_clube_visitante_nome,
    partida_placar_mandante,
    partida_placar_visitante
FROM silver.cartola_lh;


"""

with engine.begin() as conn:
    conn.execute(text(ddl_dim_partida))

print("gold.dim_partida criada/populada")


In [ ]:
ddl_fato = """
DROP TABLE IF EXISTS gold.f_cartola_pontuacao CASCADE;

CREATE TABLE gold.f_cartola_pontuacao (
    fact_id                              BIGSERIAL PRIMARY KEY,
    atleta_sk                            BIGINT NOT NULL,
    clube_sk                             BIGINT,
    partida_sk                           BIGINT NOT NULL,
    temporada                            INTEGER,
    as_partida_casa_fora                 TEXT,

    scout_media_num                      DOUBLE PRECISION,
    scout_preco_num                      DOUBLE PRECISION,
    scout_pontos_num                    DOUBLE PRECISION,
    scout_variacao_num                   DOUBLE PRECISION,
    scout_ponto_por_falta_sofrida        BIGINT,
    scout_ponto_por_passe_errado         BIGINT,
    scout_ponto_por_assistencia          BIGINT,
    scout_ponto_por_finalizacao_na_trave BIGINT,
    scout_ponto_por_finalizacao_defendida BIGINT,
    scout_ponto_por_finalizacao_para_fora BIGINT,
    scout_ponto_por_gol                  BIGINT,
    scout_ponto_por_impedimento          BIGINT,
    scout_ponto_por_penalti_perdido      BIGINT,
    scout_ponto_por_roubada_de_bola      BIGINT,
    scout_ponto_por_falta_cometida       BIGINT,
    scout_ponto_por_gol_contra           BIGINT,
    scout_ponto_por_cartao_amarelo       BIGINT,
    scout_ponto_por_cartao_vermelho      BIGINT,
    scout_ponto_por_nao_sofrer_gol       BIGINT,
    scout_ponto_por_defesa_dificil       BIGINT,
    scout_ponto_por_defesa_de_penalti    BIGINT,
    scout_ponto_por_gol_sofrido          BIGINT
);

INSERT INTO gold.f_cartola_pontuacao (
    atleta_sk,
    clube_sk,
    partida_sk,
    temporada,
    as_partida_casa_fora,
    scout_media_num,
    scout_preco_num,
    scout_pontos_num,
    scout_variacao_num,
    scout_ponto_por_falta_sofrida,
    scout_ponto_por_passe_errado,
    scout_ponto_por_assistencia,
    scout_ponto_por_finalizacao_na_trave,
    scout_ponto_por_finalizacao_defendida,
    scout_ponto_por_finalizacao_para_fora,
    scout_ponto_por_gol,
    scout_ponto_por_impedimento,
    scout_ponto_por_penalti_perdido,
    scout_ponto_por_roubada_de_bola,
    scout_ponto_por_falta_cometida,
    scout_ponto_por_gol_contra,
    scout_ponto_por_cartao_amarelo,
    scout_ponto_por_cartao_vermelho,
    scout_ponto_por_nao_sofrer_gol,
    scout_ponto_por_defesa_dificil,
    scout_ponto_por_defesa_de_penalti,
    scout_ponto_por_gol_sofrido
)
SELECT
    da.atleta_sk,
    dc.clube_sk,
    dp.partida_sk,
    s.temporada,
    s.as_partida_casa_fora,
    s.scout_media_num,
    s.scout_preco_num,
    s.scout_pontos_num,
    s.scout_variacao_num,
    s.scout_ponto_por_falta_sofrida,
    s.scout_ponto_por_passe_errado,
    s.scout_ponto_por_assistencia,
    s.scout_ponto_por_finalizacao_na_trave,
    s.scout_ponto_por_finalizacao_defendida,
    s.scout_ponto_por_finalizacao_para_fora,
    s.scout_ponto_por_gol,
    s.scout_ponto_por_impedimento,
    s.scout_ponto_por_penalti_perdido,
    s.scout_ponto_por_roubada_de_bola,
    s.scout_ponto_por_falta_cometida,
    s.scout_ponto_por_gol_contra,
    s.scout_ponto_por_cartao_amarelo,
    s.scout_ponto_por_cartao_vermelho,
    s.scout_ponto_por_nao_sofrer_gol,
    s.scout_ponto_por_defesa_dificil,
    s.scout_ponto_por_defesa_de_penalti,
    s.scout_ponto_por_gol_sofrido
FROM silver.cartola_lh s
JOIN gold.dim_atleta  da ON da.atleta_id = s.atleta_id
LEFT JOIN gold.dim_clube   dc ON dc.clube_id = s.clube_id
JOIN gold.dim_partida dp ON dp.partida_id = s.partida_id
                        AND dp.temporada  = s.temporada;
"""

with engine.begin() as conn:
    conn.execute(text(ddl_fato))

print("gold.f_cartola_pontuacao criada/populada")
